In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
from collections import defaultdict
from itertools import combinations

In [16]:
with open('./data/2016_q1_newman-neighbours_kcore.pickle', 'rb') as picklefile:
    nbrs = pickle.load(picklefile)
with open('./data/2016_q1_newman-weights_kcore.pickle', 'rb') as picklefile:
    newman_weights = pickle.load(picklefile)
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    cci_weights = pickle.load(picklefile, encoding='latin1')

In [21]:
cci_weights_dict = defaultdict(float)
for item in cci_weights:
    cci_weights_dict[item[0]] = item[1]

In [3]:
len(weights)

1058308

In [4]:
len(nbrs)

3053

In [5]:
#map ids to 0-3052
keys = range(len(nbrs))
values = sorted(nbrs.keys())
ids = dict(zip(keys,values))

In [17]:
def adjacency_matrix(nbrs, weights):
    #map ids to 0-3052
    keys = range(len(nbrs))
    values = sorted(nbrs.keys())
    ids = dict(zip(keys,values))

    #weighted adjacency matrix
    A = np.zeros((len(ids),len(ids)))
    w = set(weights.keys())
    pairs = combinations(keys, 2) 
    for u,v in pairs:  #u<v
        if ((ids[u],ids[v])in w)or((ids[v],ids[u])in w):
            A[u,v] = weights[(ids[u],ids[v])]
            A[v,u] = weights[(ids[u],ids[v])]

    #Unweighted adjacency matrix
    A_un = np.zeros((len(ids),len(ids)))
    w = set(weights.keys())
    pairs = combinations(keys, 2) 
    for u,v in pairs:  #u<v
        if ((ids[u],ids[v])in w)or((ids[v],ids[u])in w):
            A_un[u,v] = 1
            A_un[v,u] = 1
    return A, A_un

In [22]:
A, A_un = adjacency_matrix(nbrs, cci_weights_dict)

In [7]:
#Unweighted adjacency matrix
A_un = np.zeros((len(ids),len(ids)))
w = set(weights.keys())
pairs = combinations(keys, 2) 
for u,v in pairs:  #u<v
    if ((ids[u],ids[v])in w)or((ids[v],ids[u])in w):
        A_un[u,v] = 1
        A_un[v,u] = 1

## Katz Score Calculation

In [1]:
def generate_katz(A, A_un, score_name):
    I = np.identity(A.shape[0])
    beta_list = [0.05, 0.005, 0.0005, 0.00005]
    beta_names = ['0_05','0_005','0_0005','0_00005']
    katz_weighted = defaultdict(float)
    katz_unweighted = defaultdict(float)

    for beta, name in zip(beta_list, beta_names):
        katz = np.linalg.inv(I-beta*A) - I
        pairs = combinations(keys, 2)
        for u,v in pairs:
            if ids[u] > ids[v]:
                katz_weighted[(ids[v],ids[u])] = katz[u][v]
            else:
                katz_weighted[(ids[u],ids[v])] = katz[u][v]
        with open('./data/{}_katz_weighted_{}.pickle'.format(score_name, name), 'wb') as picklefile:
            pickle.dump(katz_weighted, picklefile)

    for beta, name in zip(beta_list, beta_names):
        katz = np.linalg.inv(I-beta*A_un) - I
        pairs = combinations(keys, 2)
        for u,v in pairs:
            if ids[u] > ids[v]:
                katz_unweighted[(ids[v],ids[u])] = katz[u][v]
            else:
                katz_unweighted[(ids[u],ids[v])] = katz[u][v]
        with open('./data/{}_katz_unweighted_{}.pickle'.format(score_name, name), 'wb') as picklefile:
            pickle.dump(katz_unweighted, picklefile)

In [23]:
generate_katz(A, A_un, "cci")